In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [ ]:
URL = "https://www.cntraveler.com/galleries/2015-11-27/the-50-most-beautiful-places-in-the-world"

In [ ]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geolocator = Nominatim(user_agent="geocode_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)

In [ ]:
webpage = requests.get(URL, headers=HEADER)
webpage

In [ ]:
soup = BeautifulSoup(webpage.content, "html.parser")
soup

In [ ]:
places_list = soup.find_all("span", attrs = {'class': 'GallerySlideCaptionHedText-iqjOmM jwPuvZ'})
#print(len(places))
#print(places[4].text)
#places.pop(4)
#print(len(places))
#print(places[4].text)
places_list[0:10]

In [ ]:
places = []
places_list.pop(4)
l = len(places_list)
l
for i in range(11):
    places.append(places_list[i].text)

places

In [ ]:
town,city,state = "","",""
location_description = geocoder.geocode(places[4].text)
try:
    town = location_description[0]['components']['town']
except:
    print ("No data")
try:
    city = location_description[0]['components']['city']
except:
    print ("No data")
try:
    state = location_description[0]['components']['state']
except:
    print ("No data")

print( town + " " + city + "  " + state + "  ")
#location_description

In [ ]:
city_address = []
for i in range(20):
    
    # GETTING RAW DATA
    try:
        location_description = geocoder.geocode(places[i].text)
    except Exception:
        print(str(i) + " " + "No data found for location: " + places[i].text)
        continue
    
    # GETTING ADDRESS VALUES
    latitude = str(location_description[0]['geometry']['lat'])
    longitude = str(location_description[0]['geometry']['lng'])
    loc_title = places[i].text.split(",")[0]
    
    try:
        city = location_description[0]['components']['city']
    except Exception:
        try:
            city = location_description[0]['components']['town']
        except Exception:
            try:
                city = location_description[0]['components']['state']
            except Exception:
                city = "NULL"
    country = location_description[0]['components']['country']
    country_code = location_description[0]['components']['country_code']
    continent = location_description[0]['components']['continent']
    
    #GETTING POSTCODE
    lat_long = str(latitude) + ","+ str(longitude) 
    location2 = geolocator.reverse(lat_long, language="en")
    try:
        postcode = location2.raw['address']['postcode'].replace(" ","")
    except:
        postcode = "NULL"
        
        
    # ADDING VARIABLES TO DICTIONARY
    #print(str(i) +"  "+ loc_title + "  " + city +"  " + country + "  " + country_code.upper() + "  " + continent + "  " + latitude + "  "+ longitude+ "  " + postcode)
    address_dict = {'Location': loc_title, 'City/State': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode}
    city_address.append(address_dict)
    
print(city_address)

In [ ]:
ca_most_beautiful_df = pd.DataFrame.from_dict(city_address)
ca_most_beautiful_df